In [241]:
## DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import warnings
from utils import data_split

# 모든 경고 메시지를 무시하고 출력하지 않음
warnings.filterwarnings("ignore")

## Model
import torch    
import torch.nn as nn
import torch.nn.functional as F
import torch.fft
from layers.Embed import DataEmbedding
from layers.Conv_Blocks import Inception_Block_V1   
            #convolution block used for convoluting the 2D time data, changeable

## Train
import time
import torch.optim as optim

### train + test(4년간 3월 평균)

In [242]:
unique_code = ['BC_C_J', 'TG_B_J', 'CR_B_J', 'RD_E_S', 'BC_A_J', 'CB_F_J', 'RD_D_J', 'TG_A_S', 'BC_E_S', 'CR_D_J', 'BC_A_S', 'BC_B_S', 'TG_E_J', 
               'CR_E_S', 'RD_F_J', 'BC_E_J', 'TG_A_J', 'CR_C_J', 'CR_D_S', 'TG_C_J', 'CB_A_S', 'TG_D_J', 'CR_E_J', 'RD_C_S', 'BC_C_S', 'CB_E_J', 
               'RD_E_J', 'BC_D_J', 'CR_A_J', 'TG_E_S', 'TG_C_S', 'TG_D_S', 'RD_A_S', 'RD_A_J', 'RD_D_S', 'TG_B_S', 'CB_D_J', 'CB_A_J', 'BC_B_J']

data = pd.read_csv('~/Developer/private/Dacon/jeju/data/train.csv')
data_list = data_split(data)

dataset = {}
march_data = {}
for code in unique_code:
    march_data[code] = {}
    march_data[code]['2019'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2019-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2019-03-31')].reset_index(drop = True)
    march_data[code]['2020'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2020-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2020-03-31')].reset_index(drop = True)
    march_data[code]['2021'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2021-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2021-03-31')].reset_index(drop = True)
    march_data[code]['2022'] = data_list[f'data_{code}'][(data_list[f'data_{code}']['timestamp'] >= '2022-03-04') & (data_list[f'data_{code}']['timestamp'] <= '2022-03-31')].reset_index(drop = True)
    

    avg_supply = []
    for i in range(28):
        supply_19 = march_data[code]['2019']['supply(kg)'][i]
        supply_20 = march_data[code]['2020']['supply(kg)'][i] 
        supply_21 = march_data[code]['2021']['supply(kg)'][i] 
        supply_22 =march_data[code]['2022']['supply(kg)'][i] 

        supplies = [supply_19, supply_20, supply_21, supply_22]
        filtered_supplies = [supply for supply in supplies if supply != 0]

        # 평균 계산
        if filtered_supplies:
            average_supply = sum(filtered_supplies) / len(filtered_supplies)
        else: 
            average_supply = 0
        
        avg_supply.append(average_supply)

    zero_sunday = [1, 8, 15, 22]
    for idx in zero_sunday:
        avg_supply[idx] = 0


    avg_price = []
    for i in range(28):
        price_19 = march_data[code]['2019']['price(원/kg)'][i]
        price_20 = march_data[code]['2020']['price(원/kg)'][i] 
        price_21 = march_data[code]['2021']['price(원/kg)'][i] 
        price_22 =march_data[code]['2022']['price(원/kg)'][i] 

        prices = [price_19, price_20, price_21, price_22]
        filtered_prices = [price for price in prices if price != 0]

        # 평균 계산
        if filtered_prices:
            average_price = sum(filtered_prices) / len(filtered_prices)
        else: 
            average_price = 0
        
        avg_price.append(average_price)
        

    zero_sunday = [1, 8, 15, 22]
    for idx in zero_sunday:
        avg_price[idx] = 0


    march_data[code]['avg_supply'] = avg_supply
    march_data[code]['avg_price'] = avg_price

    # 2023년 3월 test용 데이터셋 생성
    new_df = march_data[code]['2019'].copy()
    # 'ID'의 연도를 2023으로 변경
    new_df['ID'] = new_df['ID'].apply(lambda x: f"{x[:5]}2023{x[9:]}")
    # 'timestamp'의 연도를 2023으로 변경
    new_df['timestamp'] = new_df['timestamp'].apply(lambda x: f"2023-{x[5:]}")
    # supply와 price의 평균값으로 변경
    new_df['supply(kg)'] = avg_supply
    new_df['price(원/kg)'] = avg_price

    march_data[code]['2023'] = new_df
    

    # 원래 데이터와 합침
    merged_df = pd.concat([data_list[f'data_{code}'], march_data[code]['2023']], axis=0, ignore_index=True)
    dataset[code] = merged_df

In [243]:
dataset['TG_A_S']

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_S_20190101,2019-01-01,TG,A,S,0.000000,0.000000
1,TG_A_S_20190102,2019-01-02,TG,A,S,0.000000,0.000000
2,TG_A_S_20190103,2019-01-03,TG,A,S,190591.000000,2526.000000
3,TG_A_S_20190104,2019-01-04,TG,A,S,137729.000000,2134.000000
4,TG_A_S_20190105,2019-01-05,TG,A,S,134039.000000,2075.000000
...,...,...,...,...,...,...,...
1546,TG_A_2023190327,2023-03-27,TG,A,S,9511.000000,5015.666667
1547,TG_A_2023190328,2023-03-28,TG,A,S,16388.000000,5254.333333
1548,TG_A_2023190329,2023-03-29,TG,A,S,16599.666667,4729.000000
1549,TG_A_2023190330,2023-03-30,TG,A,S,12938.125000,4760.250000


### DataLoader

In [244]:
seq_len = 64
label_len = 28
pred_len = 28



type_map = {'train': 0, 'val': 1, 'test': 2}
flag = 'train'
set_type = type_map[flag]
features = 'S' # single or multi
target = 'price(원/kg)' 
scale = True
timeenc = 0 # time_feature가 존재하는지 여부 : 없으면 임의로 생성해주지만 우리 데이터에는 존재함
freq = 'd'

In [245]:
class Dataset_jeju(Dataset):
        def __init__(self, flag='train', size=[64,28,28],
             features='S', scale=True, timeenc=0, freq='d', seasonal_patterns=None):
            # size [seq_len, label_len, pred_len]
            # info
            if size == None:
                self.seq_len = 24 * 4 * 4 # 16일 데이터를 이용해 
                self.label_len = 24 * 4 
                self.pred_len = 24 * 4 # 4일간의 데이터 예측? 28
            else:
                self.seq_len = size[0]
                self.label_len = size[1]
                self.pred_len = size[2]
            # init
            assert flag in ['train', 'test', 'val']
            type_map = {'train': 0, 'val': 1, 'test': 2}
            self.set_type = type_map[flag]
            self.features = features
            self.target = target
            self.scale = scale
            self.timeenc = timeenc
            self.freq = freq

            self.train_mean = None
            self.train_std = None

            # After initialization, call __read_data__() to manage the data file.
            self.__read_data__()

        def __read_data__(self):
                self.scaler = StandardScaler()

                #get raw data from path
                df_raw = dataset['TG_B_J']

                # split data set into train, vali, test. border1 is the left border and border2 is the right.
                # Once flag(train, vali, test) is determined, __read_data__ will return certain part of the dataset.
                border1s = [0, 1350 - seq_len, 1495 - seq_len]
                border2s = [1350, 1495, 1523]
                border1 = border1s[self.set_type]
                border2 = border2s[self.set_type]

                #decide which columns to select
                if self.features == 'M' or self.features == 'MS':
                        cols_data = df_raw.columns[1:] # column name list (remove 'date')
                        df_data = df_raw[cols_data]  #remove the first column, which is time stamp info
                elif self.features == 'S':
                        df_data = df_raw[[self.target]] # target column

                #scale data by the scaler that fits training data
                if self.scale:
                        train_data = df_data[border1s[0]:border2s[0]]
                        #train_data.values: turn pandas DataFrame into 2D numpy
                        self.train_mean = train_data.mean().values
                        self.train_std = train_data.std().values
                        self.scaler.fit(train_data.values)  
                        data = self.scaler.transform(df_data.values)
                else:
                        data = df_data.values 
                
                # 날짜를 년/월/일/요일 형태로 자르고 리스트로 변환
                # [[2019    1    1    1]
                #  [2019    1    2    2]
                #  [2019    1    3    3]
                #  ...
                
                df_stamp = df_raw[['timestamp']][border1:border2]
                df_stamp['timestamp'] = pd.to_datetime(df_stamp.timestamp) 

                if self.timeenc == 0:  #time feature encoding is fixed or learned
                        # df_stamp['year'] = df_stamp.timestamp.apply(lambda row: row.year, 1)
                        df_stamp['month'] = df_stamp.timestamp.apply(lambda row: row.month, 1)
                        df_stamp['day'] = df_stamp.timestamp.apply(lambda row: row.day, 1)
                        df_stamp['weekday'] = df_stamp.timestamp.apply(lambda row: row.weekday(), 1)
                        
                        #now df_frame has multiple columns recording the month, day etc. time stamp
                        # next we delete the 'date' column and turn 'DataFrame' to a list
                        data_stamp = df_stamp.drop(['timestamp'], axis = 1).values

                # elif self.timeenc == 1: #time feature encoding is timeF
                #         data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
                #         data_stamp = data_stamp.transpose(1, 0)
                        
                
                # data_x and data_y are same copy of a certain part of data
                self.data_x = data[border1:border2]
                self.data_y = data[border1:border2]
                self.data_stamp = data_stamp

        def __getitem__(self, index):
                #given an index, calculate the positions after this index to truncate the dataset
                s_begin = index
                s_end = s_begin + self.seq_len
                r_begin = s_end - self.label_len
                r_end = r_begin + self.label_len + self.pred_len

                #input and output sequence
                seq_x = self.data_x[s_begin:s_end]
                seq_y = self.data_y[r_begin:r_end]

                #time mark
                seq_x_mark = self.data_stamp[s_begin:s_end]
                seq_y_mark = self.data_stamp[r_begin:r_end]

                return seq_x, seq_y, seq_x_mark, seq_y_mark, self.train_mean, self.train_std

        def __len__(self):
                return len(self.data_x) - self.seq_len - self.pred_len + 1

In [246]:
from torch.utils.data import DataLoader

# 데이터셋 객체 생성
dataset_train = Dataset_jeju(flag='train', size=[64, 28, 28], features='S', scale=True, timeenc=0, freq='h')
dataset_val = Dataset_jeju(flag='val', size=[64, 28, 28], features='S', scale=True, timeenc=0, freq='h')
dataset_test = Dataset_jeju(flag='test', size=[64, 28, 28], features='S', scale=True, timeenc=0, freq='h')

# 데이터로더 생성
dataloader_train = DataLoader(dataset_train, batch_size=16, shuffle=True, drop_last = True)
dataloader_val = DataLoader(dataset_val, batch_size=16, shuffle=True, drop_last = True)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, drop_last = False)

### Model code

In [247]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import math


class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()

class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='d'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13
        year_size = 6

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)
        # self.year_embed = Embed(year_size, d_model)

    def forward(self, x):
        x = x.long()
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])
        # year_x = self.year_embed(x[:, :, 0])

        # return year_x + weekday_x + day_x + month_x 
        return weekday_x + day_x + month_x 


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) 
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x) + self.position_embedding(x)
        else:
            x = self.value_embedding(
                x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)


def FFT_for_Period(x, k=5):
    # [B, T, C]
    xf = torch.fft.rfft(x, dim=1)
    # find period by amplitudes
    frequency_list = abs(xf).mean(0).mean(-1)
    frequency_list[0] = 0
    _, top_list = torch.topk(frequency_list, k)
    top_list = top_list.detach().cpu().numpy()
    period = x.shape[1] // top_list
    return period, abs(xf).mean(-1)[:, top_list]


class TimesBlock(nn.Module):
    def __init__(self, configs):
        super(TimesBlock, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.k = configs.top_k
        # parameter-efficient design
        self.conv = nn.Sequential(
            Inception_Block_V1(configs.d_model, configs.d_ff,
                               num_kernels=configs.num_kernels),
            nn.GELU(),
            Inception_Block_V1(configs.d_ff, configs.d_model,
                               num_kernels=configs.num_kernels)
        )

    def forward(self, x):
        B, T, N = x.size()
        period_list, period_weight = FFT_for_Period(x, self.k)

        res = []
        for i in range(self.k):
            period = period_list[i]
            # padding
            if (self.seq_len + self.pred_len) % period != 0:
                length = (
                                 ((self.seq_len + self.pred_len) // period) + 1) * period
                padding = torch.zeros([x.shape[0], (length - (self.seq_len + self.pred_len)), x.shape[2]]).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = (self.seq_len + self.pred_len)
                out = x
            # reshape
            out = out.reshape(B, length // period, period,
                              N).permute(0, 3, 1, 2).contiguous()
            # 2D conv: from 1d Variation to 2d Variation
            out = self.conv(out)
            # reshape back
            out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
            res.append(out[:, :(self.seq_len + self.pred_len), :])
        res = torch.stack(res, dim=-1)
        # adaptive aggregation
        period_weight = F.softmax(period_weight, dim=1)
        period_weight = period_weight.unsqueeze(
            1).unsqueeze(1).repeat(1, T, N, 1)
        res = torch.sum(res * period_weight, -1)
        # residual connection
        res = res + x
        return res


class Model(nn.Module):

    def __init__(self, configs):
        super(Model, self).__init__()
        self.configs = configs
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.label_len = configs.label_len
        self.pred_len = configs.pred_len
        self.model = nn.ModuleList([TimesBlock(configs)
                                    for _ in range(configs.e_layers)])
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq,
                                           configs.dropout)
        self.layer = configs.e_layers
        self.layer_norm = nn.LayerNorm(configs.d_model)
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            self.predict_linear = nn.Linear(
                self.seq_len, self.pred_len + self.seq_len)
            self.projection = nn.Linear(
                configs.d_model, configs.c_out, bias=True)


    def forecast(self, x_enc, x_mark_enc):
        # Normalization from Non-stationary Transformer
        means = x_enc.mean(1, keepdim=True).detach()
        x_enc = x_enc - means
        stdev = torch.sqrt(
            torch.var(x_enc, dim=1, keepdim=True, unbiased=False) + 1e-5)
        x_enc /= stdev

        # embedding
        enc_out = self.enc_embedding(x_enc, x_mark_enc)  # [B,T,C]
        enc_out = self.predict_linear(enc_out.permute(0, 2, 1)).permute(
            0, 2, 1)  # align temporal dimension
        # TimesNet
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))
        # porject back
        dec_out = self.projection(enc_out)

        # De-Normalization from Non-stationary Transformer
        dec_out = dec_out * \
                  (stdev[:, 0, :].unsqueeze(1).repeat(
                      1, self.pred_len + self.seq_len, 1))
        dec_out = dec_out + \
                  (means[:, 0, :].unsqueeze(1).repeat(
                      1, self.pred_len + self.seq_len, 1))
        return dec_out


    def forward(self, x_enc, x_mark_enc):
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            dec_out = self.forecast(x_enc, x_mark_enc)
            return dec_out[:, -self.pred_len:, :]  # [B, L, D]

### Configs

In [248]:
class Configs:
    def __init__(self):
        self.seq_len = 64
        self.label_len = 28
        self.pred_len = 28
        self.top_k = 5
        self.d_model = 32
        self.d_ff = 32
        self.num_kernels = 3
        self.e_layers = 2
        self.d_layers = 1
        self.enc_in = 1
        self.dec_in = 1
        self.c_out = 1
        self.embed = 16
        self.freq = 'd'
        self.dropout = 0.1
        self.task_name = 'short_term_forecast'
        self.c_out = 1
        self.seasonal_patternes = 'Monthly'
        self.features = 'S'

configs = Configs()

### EarlyStopping

In [249]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience # how many times will you tolerate for loss not being on decrease
        self.verbose = verbose  # whether to print tip info
        self.counter = 0 # now how many times loss not on decrease
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)

        # meaning: current score is not 'delta' better than best_score, representing that 
        # further training may not bring remarkable improvement in loss. 
        elif score < self.best_score + self.delta:  
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            # 'No Improvement' times become higher than patience --> Stop Further Training
            if self.counter >= self.patience:
                self.early_stop = True

        else: #model's loss is still on decrease, save the now best model and go on training
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
    ### used for saving the current best model
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path + '/' + 'checkpoint.pth')
        self.val_loss_min = val_loss

### Train

In [250]:
def vali(model, vali_loader, criterion):
        total_loss = []

        #evaluation mode
        model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark,_,_) in enumerate(vali_loader):
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float()

                batch_x_mark = batch_x_mark.float().to(device)
                batch_y_mark = batch_y_mark.float().to(device)

                
                outputs = model(batch_x, batch_x_mark)
                f_dim = 0
                outputs = outputs[:, -pred_len:, f_dim:]
                batch_y = batch_y[:, -pred_len:, f_dim:].to(device)

                pred = outputs.detach().cpu()
                true = batch_y.detach().cpu()

                loss = criterion(pred, true)

                total_loss.append(loss)
        total_loss = np.average(total_loss)
        model.train()
        return total_loss

In [254]:
###########
model = Model(configs)
epochs = 10
learning_rate = 0.05

seq_len = 64
pred_len = 28
label_len = 28
################


train_loader = dataloader_train
vali_loader = dataloader_val
test_loader = dataloader_test

_, _, _, _, test_mean, test_std = next(iter(dataloader_test))


# set path of checkpoint for saving and loading model
path = 'Savemodel'
time_now = time.time()

train_steps = len(train_loader)

# EarlyStopping is typically a custom class or function that monitors the performance 
# of a model during training, usually by tracking a certain metric (commonly validation 
# loss or accuracy).It's a common technique used in deep learning to prevent overfitting 
# during the training
early_stopping = EarlyStopping(patience=3, verbose=True)

#Optimizer and Loss Function Selection
model_optim = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

for epoch in range(epochs):
    iter_count = 0
    train_loss = []
    model.train()
    epoch_time = time.time()

    #begin training in this epoch
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark,_,_) in enumerate(train_loader):
        iter_count += 1
        model_optim.zero_grad()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        batch_x = batch_x.float().to(device)  #input features
        batch_y = batch_y.float().to(device)  #target features

        # _mark holds information about time-related features. Specifically, it is a 
        # tensor that encodes temporal information and is associated with the 
        # input data batch_x.
        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        
        outputs = model(batch_x, batch_x_mark)
        f_dim = 0 
        # f_dim = -1 if args.features == 'MS' else 0
        outputs = outputs[:, -pred_len:, f_dim:]
        batch_y = batch_y[:, -pred_len:, f_dim:].to(device)
        loss = criterion(outputs, batch_y)
        train_loss.append(loss.item())

        # When train rounds attain some 100-multiple, print speed, left time, loss. etc feedback
        if (i + 1) % 100 == 0:
            print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
            speed = (time.time() - time_now) / iter_count
            left_time = speed * ((epochs - epoch) * train_steps - i)
            print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
            iter_count = 0
            time_now = time.time()

        # #BP
        # if args.use_amp:
        #     scaler.scale(loss).backward()
        #     scaler.step(model_optim)
        #     scaler.update()
        # else:
        #     loss.backward()
        #     model_optim.step()
        loss.backward()
        model_optim.step()

    
    #This epoch comes to end, print information
    print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
    train_loss = np.average(train_loss)

    #run test and validation on current model
    vali_loss = vali(model, vali_loader, criterion)
    test_loss = vali(model, test_loader, criterion)

    #print train, test, vali loss information
    print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
        epoch + 1, train_steps, train_loss, vali_loss, test_loss))
    
    #Decide whether to trigger Early Stopping. if early_stop is true, it means that 
    #this epoch's training is now at a flat slope, so stop further training for this epoch.
    early_stopping(vali_loss, model, path)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    # #adjust learning keys
    # adjust_learning_rate(model_optim, epoch + 1, args)

# best_model_path = path + '/' + 'checkpoint.pth'

# # loading the trained model's state dictionary from a saved checkpoint file 
# # located at best_model_path.
# model.load_state_dict(torch.load(best_model_path))

preds = []
trues = []
model.eval()
with torch.no_grad():
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark,_,_) in enumerate(test_loader):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)

        
        outputs = model(batch_x, batch_x_mark)
        print(outputs.shape)
        print('A')

        f_dim = 0
        outputs = outputs[:,-pred_len:, f_dim:]
        batch_y = batch_y[:,-pred_len:, f_dim:].to(device)
    
        outputs = outputs.detach().cpu().numpy()
        batch_y = batch_y.detach().cpu().numpy()

        #inverse the data if scaled
        
        outputs = outputs * test_std[0].item() + test_mean[0].item()
        batch_y = batch_y * test_std[0].item() + test_mean[0].item()
  
        pred = outputs#.view(-1).numpy()
        true = batch_y

        preds.append(pred)
        trues.append(true)

       
preds = np.array(preds)
trues = np.array(trues)  # shape[batch_num, batch_size, pred_len, features]
print('test shape:', preds.shape, trues.shape)
preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
print('test shape:', preds.shape, trues.shape)
    
        



Epoch: 1 cost time: 6.630518198013306
Epoch: 1, Steps: 78 | Train Loss: 0.9745246 Vali Loss: 0.2974105 Test Loss: 0.4719639
Validation loss decreased (inf --> 0.297411).  Saving model ...
Epoch: 2 cost time: 5.505518913269043
Epoch: 2, Steps: 78 | Train Loss: 0.7376696 Vali Loss: 0.1657926 Test Loss: 0.3786559
Validation loss decreased (0.297411 --> 0.165793).  Saving model ...
Epoch: 3 cost time: 5.489583730697632
Epoch: 3, Steps: 78 | Train Loss: 0.7003860 Vali Loss: 0.1192132 Test Loss: 0.3251331
Validation loss decreased (0.165793 --> 0.119213).  Saving model ...
Epoch: 4 cost time: 5.642007112503052
Epoch: 4, Steps: 78 | Train Loss: 0.6836522 Vali Loss: 0.1676301 Test Loss: 0.3283365
EarlyStopping counter: 1 out of 3
Epoch: 5 cost time: 5.747663259506226
Epoch: 5, Steps: 78 | Train Loss: 0.6743232 Vali Loss: 0.1284957 Test Loss: 0.3189631
EarlyStopping counter: 2 out of 3
Epoch: 6 cost time: 5.85167384147644
Epoch: 6, Steps: 78 | Train Loss: 0.6524324 Vali Loss: 0.1199608 Test Los

In [253]:
print(preds)

[[[1506.7806  ]
  [-491.20166 ]
  [1751.3953  ]
  [1651.3871  ]
  [2577.1685  ]
  [2423.4749  ]
  [2509.41    ]
  [1809.4152  ]
  [  72.426025]
  [2460.4446  ]
  [2623.8328  ]
  [3206.3164  ]
  [3100.8818  ]
  [2975.5361  ]
  [2771.7603  ]
  [  96.093506]
  [3603.1216  ]
  [3434.037   ]
  [3576.2192  ]
  [3613.2058  ]
  [3407.312   ]
  [3305.0933  ]
  [ 660.0215  ]
  [3636.4763  ]
  [3040.2854  ]
  [3773.808   ]
  [3439.8186  ]
  [3375.7178  ]]]
